### Automated accuracy testing

This is script is made for testing Renegade with various suites.  
Particularly: http://www.bergbomconsulting.se/chess/epdfiles/wac.epd

As long as the engine returns `bestmove ....` and then sends a stopping message (in this case `Stopping engine.`), it should be easy to adapt it to other engines as well.

Requires the `python-chess` library for converting the algebraic notation to uci string.

In [ ]:
# Set the path to executable and json here
path = "../x64/Release/Renegade.exe"
epd = "wac.epd"

parameters = "movetime 1000"

In [ ]:
import subprocess
import json
import time
import chess

tests = []
with open(epd) as file:
    lines = [line.rstrip().split() for line in file]
    for line in lines:
        fen = line[0] + " " + line[1] + " " + line[2] + " " + line[3] + " 0 1"
        moves = []
        i = line.index("bm")
        while True:
            i += 1
            best_move = line[i]
            if best_move.endswith(";"):
                best_move = best_move[:-1]
                moves.append(best_move)
                break
            moves.append(best_move)
        tests.append({"fen": fen, "bm": moves})
    
def run():
    print("Started.\n")
    good = 0
    total = 0
    for test in tests:
        total += 1
        board = chess.Board(test['fen'])
        bm_uci = []
        for bm in test['bm']:
            bm_uci.append(board.parse_san(bm).uci())
        output = runtest(test['fen'])
        if str(output, 'ascii') in bm_uci:
            good += 1
        print('%3d. %-68s ' % (total, test['fen']), '   -> ', output, '(correct: ', bm_uci, ')')
    print("\nCompleted.")
    print("Constraints: '%s', correct: %d/%d" % (parameters, good, total))

def runtest(fen):
    proc = subprocess.Popen([path, '-l', ''], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
    proc.stdin.write(('position fen ' + fen + '\n').encode())
    proc.stdin.write(('go %s\n' % parameters).encode())
    proc.stdin.close()
    i = 0
    start = time.time_ns()
    lastline = ""
    while True:
        out = proc.stdout.readline().strip()
        if out == b'Stopping engine.': break
        lastline = out
    return lastline.split()[1]

In [ ]:
run()